# Generating RCompanion notebooks from extracted markdown via notedown

Follows from `one`.

Meant to be run in sessions launched from [here](https://github.com/fomightez/rcomp_testenv). 

## Preparation

### STEP #1: Get this notebook running in a freshly-launched Binder session 

**WHERE TO RUN:**  
Run this in a Binder session launched from anywhere since this doesn't include any fancy dependencies not handled by pip.



-----






In [ ]:
# NOT NEEDED HERE. FROM SOURCE NOTEBOOK. KEEPING FOR NOW BECAUSE MAY EVENTUALLY SOMETHING SIMILAR(?):
#
'''
# For Clearing out the contents presently there so things aren't as cluttered. 
# But don't delete this notebook.
from shlex import quote
pathname_of_file_to_keep = quote("notebooks/Generating later Circos tutorial notebooks from extracted markdown via notedown and papermill.ipynb")
name_of_file_to_keep = quote("Generating later Circos tutorial notebooks from extracted markdown via notedown and papermill.ipynb")

# based on Olivier Dulac's comment at https://unix.stackexchange.com/questions/153862/remove-all-files-directories-except-for-one-file
%cd ..
!cp $pathname_of_file_to_keep .
!rm -rf notebooks
!mkdir notebooks
!mv $name_of_file_to_keep notebooks/
%cd notebooks
''';

### STEP #2: Unpack previous results

**REQUIRED FILES**

Besides this notebook (and installed notedown packages) the following must be uploaded to the running binder session:

- `FirstSetmarkdown_from_RCompanion.tar.gz` from `Getting R Companion html and converting to markdown text.ipynb` . This contains markdowns made for each of the pages, plus two pertinent lists stored as json.

Unpack that. And bring in the list of urls and files into memory.

In [ ]:
import os
unpacked_example = os.path.join(".","RCompanion_urls_to_get.json")
file_needed = "FirstSetmarkdown_from_RCompanion.tar.gz"

import sys
if os.path.isfile(unpacked_example):
    sys.stderr.write("\nAppears '{}' has already been unpacked.\n".format(file_needed))
elif os.path.isfile(file_needed):
    !tar xzf {file_needed}
else:
    sys.stderr.write("\n\n*****************ERROR**************************\n"
        "The file '{0}' is needed.\n"
        "Upload '{0}' to this Jupyter session and re-run this cell.\n"
        "*****************ERROR**************************\n".format(file_needed))
    sys.exit(1)
stored_json_files_and_names = [
    "RCompanion_markdowns_made.json",
    "RCompanion_urls_to_get.json"
                            ]
stored_json_files_and_names_to_unpack_as = {k:f'{k.split("_")[0]}_{k.split("_")[1]}' for k in stored_json_files_and_names}
import json
for k,v in stored_json_files_and_names_to_unpack_as.items():
    with open(k, 'r') as f:
        globals()[v] = json.load(f)

**REQUIRED MODULES/PACKAGES**
Install notedown.

In [ ]:
#!pip install papermill
!pip install notedown

-----



## STEP #3: Step through the markdown files and fix some issues:

List of fixes:

- fix `b_10.md` to look like the other pages in regard to lines that begin `title: '`.<--- I think altering how handling the spaces has made this moot now. But it looks like I may need to address the doubling of the dashes if that matters.
- remove most everything except what is marked as `::: {#content}`, marked by ending at `::: {#footer}`. The only other stuff to keep is what is right near the top on the line that begins `title: '` (maked by having `'` after it), the  line after `{#mastword}` (marked by having `Salvatore S. Mangiafico` after it), and the two lines after `::: {#title}`(marked by having `:::` after it)..<--- I think altering how handling the spaces has made this need to be altered, too.
- I'd like the headings which currently have string starting `_toc` on the lines be level theee headings, i.e., have `###`.
- remove any lines that are just ":::" if strip away empty space.
- remove lines reading 'Advertisement' or starting with 'medianet'.
- remove occurence double-dashes as long as not a long string of dashes or "pp." on the line.
- replace non-breaking spaces, and spaces in results lines, that where filled with placeholder and not spaces so wouldn't introduce line breaks when pandoc converted (seems pandoc doesn't like numbers in line with spaces and makes a new line?)
- finalize results lines
- put code fences where they belong
- convert `###\n--------------------------------------------------------------` to 
`###--------------------------------------------------------------`. In other words remove the line break after the three asterisks. <--- actually now handled earlier (and differenly) and more generally by `avoid_line_breaks_caused_by_comments()`
- lines beginning `ASxSPACE###` get changed to beginning with `###`. Mainly pertinent to lines like, ``### --------------------------------------------------------------`

(**Note for trouble-shooting this section and all after:  
Each time I want to redo the next cell and ones after, the easiest way to get back to this point was to first make a separate directory below the one with this notebook called `unpacked_original` where I uploaded `FirstSetmarkdown_from_RCompanion.tar.gz` and then unpacked it with `tar -xzf FirstSetmarkdown_from_RCompanion.tar.gz`. Then to reset, I just ran in a terminal `cp *.md ../.` where the terminal working directory was presently `unpacked_original`.)

In [ ]:
repo_name = "??????????"


def fix_first_title_text(md):
    '''
    fix extracted markdown to look like the other pages in regard to lines that begin `title: '`
    '''
    with open(md, 'r') as input:
        all_md=input.read()
    new_title_line = all_md.split("title: |")[1].split("---")[0].strip().replace("\n"," ").replace("     "," ")
    first_pt = all_md.split("title: |")[0]
    the_last_pt = all_md.split("title: |")[1].split("---",1)[1]
    new_md = f"{first_pt}title: '{new_title_line}'\n{the_last_pt}" 
    return new_md

def reduce_to_content(md):
    '''
    Reduces to just content section except for keeping titles.
    
    Adds a footer placeholder for use later.
    '''
    with open(md, 'r') as input:
        all_md=input.read()
    #get three things I need and then append to contents section
    first_pt = all_md.split("title: '")[1].split("'")[0].strip()
    first_pt += "\n=============================\n\n"
    '''
    second_pt = all_md.split("{#mastword}")[1].split("Salvatore S. Mangiafico")[0].strip()
    second_pt+= "\n\nby Salvatore S. Mangiafico\n\n"
    '''
    second_pt = ("An R Companion for the Handbook of Biological Statistics\n--"
                 "-------------------------------------------------------------------")
    second_pt+= "\n\nby Salvatore S. Mangiafico\n\n"
    '''
    third_pt = all_md.split("::: {#title}")[1].split(":::")[0].strip()
    third_pt += "\n\n"
    
    content = all_md.split("::: {#content}")[1].split("::: {#footer}")[0].strip()
    '''
    content = all_md.split(
        "Summary and Analysis of Extension Program Evaluation in R")[1].split(
        "©2015 by Salvatore S. Mangiafico")[0].strip()
    footer_placeholder = "\n RCOMPANION_FOOTER_GOES_HERE"
    #new_md = f"{first_pt}{second_pt}{third_pt}{content}{footer_placeholder}" 
    new_md = f"{first_pt}{second_pt}{content}\n{footer_placeholder}" 
    return new_md


def reformat_toc_headings(md):
    '''
    Takes a markdown text file and converts the _toc headings 
    to leavel three markdown hardings
    so that they will be rendered as headings that jupyter-book can use.
    
    
    Examples:
    Inputs:
    ### []{#_Toc420844544}[Post-hoc tests]{#_Toc411541425}
    [Null hypothesis]{#_Toc411541423}
    #### [Post-hoc pairwise G-tests with]{#_Toc420844545} [RVAideMemoire]{style="font-style:normal"}
    
    Outputs:
    ### Post-hoc tests ###
    ### Null hypothesis ###
    ### Post-hoc pairwise G-tests with RVAideMemoire ###
    
    Seems regex `\[.*?\]` is what I need according to https://www.regextester.com/97589
    
    Returns the updated text for replacing the original markdown file. The reason
    I don't replace in the function is because it seems `%store` is line magic and
    cannot access the variables defined locally within a function even if
    invoked in a function. Although I could do it if I made it a global variable like I 
    somtimes d, but doing it this way since based on older code.
    '''
    new_md = ""
    with open(md, 'r') as input:
        for line in input:
            # While have lines going to remove front space from some of
            # the lines that beging with multiple pound symbols
            if line.startswith(r'ASxSPACE###'):
                line = line[len('ASxSPACE'):]
            if (("#_Toc") in line):
                bracketed_text = ' '.join(re.findall('\[(.*?)\]', line))
                new_md += f"### {bracketed_text} ###"
            else:
                new_md += line
    return new_md

def remove_triple_colon_lines(md):
    '''
    Remove any lines that are just ":::" if strip away empty space.
    
    While at that, also remove any lines reading 
    'Advertisement' or starting with 'medianet'.
    
    Also in this one handle double-dashes as long as not a long string of dashes or "pp." on the line.
    '''
    new_md = ""
    with open(md, 'r') as input:
        for line in input:
            if "--" in line and "----------" not in line and "pp." not in line:
                line = line.replace('--','-')
            if line.strip() == ":::" or (
                line.strip() == "Advertisement") or (
                line.strip().startswith("medianet")):
                continue
            else:
                new_md += line

    return new_md

def put_footer_in_placefolder(md,footer_text):
    '''
    Puts footer text into placeholder from earlier.
    '''
    footer_placeholder = r"\n \n RCOMPANION_FOOTER_CODE_GOES_HERE"
    with open(md, 'r') as input:
        all_md=input.read()
    new_md = all_md.replace(footer_placeholder,footer_text)
    return new_md

def put_in_code_fences(md):
    '''
    put in code fences
    '''
    start_placeholder_tag = u"RCOMPxCODExFENCExSTARTSxHERE"
    end_placeholder_tag = u"RCOMPxCODExFENCExENDSxHERE"
    R_code_fence_start = "```{.python .input}"
    code_fence_end = "```"
    with open(md, 'r') as input:
        all_md=input.read()
    new_md = all_md.replace(start_placeholder_tag,R_code_fence_start)
    new_md = new_md.replace(end_placeholder_tag,code_fence_end)
    new_md = new_md.replace(r'#     #     #',' ') # remove `#     #     #` occurrences because have cell borders now
    new_md = new_md.replace(r'#ASxSPACEASxSPACEASxSPACEASxSPACE #ASxSPACEASxSPACEASxSPACEASxSPACE #',' ') # remove `#     #     #` occurrences because have cell borders now
    new_md = new_md.replace(r'#ASxSPACEASxSPACEASxSPACEASxSPACEASxSPACE#ASxSPACEASxSPACEASxSPACEASxSPACEASxSPACE#',' ') # remove `#     #     #` occurrences because have cell borders now
    #while changing code blocks, remove the leading or trailing blank lines
    without_blank_lines = ""
    for p in new_md.split(R_code_fence_start):
        if code_fence_end in p:
            code_and_not_code_parts = p.split(code_fence_end)
            if code_and_not_code_parts[0].strip().endswith(r'\n    \n```'):
                without_blank_lines += (R_code_fence_start + "\n" +
                                        code_and_not_code_parts[0].strip()[:-len(r'\n    \n```')] + 
                                         "\n" + code_fence_end + "\n" + 
                                        code_and_not_code_parts[1] )
            elif code_and_not_code_parts[0].strip().endswith(r'   \n\n \n```'):
                without_blank_lines += (R_code_fence_start + "\n" +
                                        code_and_not_code_parts[0].strip()[:-len(r'   \n\n \n```')] + 
                                         "\n" + code_fence_end + "\n" + 
                                        code_and_not_code_parts[1] )
            else:
                without_blank_lines += (R_code_fence_start + "\n" +
                                        code_and_not_code_parts[0].strip() + 
                                         "\n" + code_fence_end + "\n" + 
                                        code_and_not_code_parts[1] )
        else:
            without_blank_lines += p
    new_md = without_blank_lines
    new_md = new_md.replace(R_code_fence_start+"\\","R_code_fence_start")
    new_md = new_md.replace(code_fence_end+"\\","code_fence_end")
    # fix any casses where the start of the code fence is not at the start of a line because it should be
    with_fences_at_start = ''
    for line in new_md.split("\n"):
        if R_code_fence_start in line and not line.strip().startswith(R_code_fence_start):
            parts_o_line = line.strip().split(R_code_fence_start)
            with_fences_at_start += parts_o_line[0] + "\n" + R_code_fence_start + "\n" + parts_o_line[1] + " ".join(parts_o_line[2:])
        else:
            with_fences_at_start += line + "\n"
    new_md = with_fences_at_start
    
    return new_md


import os
import sys
import re

'''
needs_fixin_title = ["b_10.md"]
for md in RCompanion_markdowns:
    if md in needs_fixin_title:
        new_md = fix_first_title_text(md)
        %store new_md > temp.txt 
        !mv temp.txt {md}
        sys.stderr.write("First title text in {} fixed"
            ".\n".format(md))
'''
for md in RCompanion_markdowns:
    print(md)
    new_md = reduce_to_content(md)
    %store new_md > temp.txt 
    !mv temp.txt {md}
    sys.stderr.write("Text extracted from {} has been reduced to content"
        ".\n".format(md))
for md in RCompanion_markdowns:
    new_md = reformat_toc_headings(md)
    %store new_md > temp.txt 
    !mv temp.txt {md}
    sys.stderr.write("Any '_toc' labeled headings in {} have been reformatted"
        ".\n".format(md))
for md in RCompanion_markdowns:
    new_md = remove_triple_colon_lines(md)
    %store new_md > temp.txt 
    !mv temp.txt {md}
    sys.stderr.write("Any lines that are just ':::' in {} have been removed"
        " and\ndouble-dashes reduced to single"
        ".\n".format(md))
    sys.stderr.write("Lines reading 'Advertisement' or starting with 'medianet'\nin {} have been removed"
        ".\n".format(md))
for md in RCompanion_markdowns:
    new_md = put_in_code_fences(md)
    %store new_md > temp.txt 
    !mv temp.txt {md}
    sys.stderr.write("Code fences have been marked in {}"
        ".\n".format(md))
start_placeholder_tag = "RCOMPxRESULTxLINExSTART"
end_placeholder_tag = "RCOMPxRESULTxLINExEND"
bracket_start_plchlder = "BRACKETxSTARTxPLCHLDR"
bracket_end_plchlder = "BRACKETxENDxPLCHLDR"
space_placeholder = "ASxSPACE"
for md in RCompanion_markdowns:
    with open(md, 'r') as input:
        all_md=input.read()
    new_md = all_md.replace(start_placeholder_tag,'<font color = "#B22222">')
    new_md = new_md.replace(end_placeholder_tag,'</font><br>')
    new_md = new_md.replace(bracket_start_plchlder,"[")
    new_md = new_md.replace(bracket_end_plchlder,"]")
    new_md = new_md.replace(space_placeholder," ")
    #new_md = new_md.replace('###\n------------------------------', '###------------------------------') # made moot by `avoid_line_breaks_caused_by_comments()` function in previous nb
    #Run a few last replace commands to lessen the amount of whitespace among the results, especially at ends (note: to get this editing
    # right, I had to run test on just single `.md` file from previous notebook to see CANNOT use raw attribute,`r`, for strings here. SEEMS
    # INCONSISTENT WHEN YOU NEED OR NOT AND HAVE TO ALWAYS TEST. (I thought I used raw for other than regex?))
    new_md = new_md.replace('\n\n<font color = "#B22222">','\n<font color = "#B22222">')
    new_md = new_md.replace('<font color = "#B22222"> </font><br>\n \n\n \n\n','\n')
    new_md = new_md.replace('<font color = "#B22222"> </font><br>\n\n\n\n\n','\n')
    new_md = new_md.replace('\n<font color = "#B22222"> </font><br>\n','\n')
    new_md = new_md.replace('</font><br>\n<font color = "#B22222"> </font><br>\n<font color = "#B22222"> </font><br>\n\n"> <font><br>\n','</font><br>\n')
    new_md = new_md.replace('<font><br>\n<font color = "#B22222"> </font><br>\n<font color = "#B22222"> </font><br>\n<font color = "#B22222"> </font><br>\n<font color = "#B22222"> </font><br>\n<font color = "#B22222"> </font><br>\n \n\n','</font><br>\n')
    new_md = new_md.replace('</font><br>\n<font color = "#B22222"></font><br>\n<font color = "#B22222"></font><br>\n \n\n','</font><br>\n')
    new_md = new_md.replace('</font><br>\n \n<font color = "#B22222"></font><br>  \n\n','</font><br>\n')
    new_md = new_md.replace('<font><br>\n<font color = "#B22222"> </font><br>\n','</font><br>\n')
    # repeat the series to catch any newer occurences made by the first round
    new_md = new_md.replace('\n\n<font color = "#B22222">',u'\n<font color = "#B22222">')
    new_md = new_md.replace('<font color = "#B22222"> </font><br>\n \n\n \n\n','\n')
    new_md = new_md.replace('<font color = "#B22222"> </font><br>\n\n\n\n\n','\n')
    new_md = new_md.replace('\n<font color = "#B22222"> </font><br>\n','\n')
    new_md = new_md.replace('</font><br>\n<font color = "#B22222"> </font><br>\n<font color = "#B22222"> </font><br>\n\n"> </font><br>\n','</font><br>\n')
    new_md = new_md.replace('</font><br>\n<font color = "#B22222"> </font><br>\n<font color = "#B22222"> </font><br>\n<font color = "#B22222"> </font><br>\n<font color = "#B22222"> </font><br>\n<font color = "#B22222"> </font><br>\n \n\n','</font><br>\n')
    new_md = new_md.replace('</font><br>\n<font color = "#B22222"></font><br>\n<font color = "#B22222"></font><br>\n \n\n','</font><br>\n')
    new_md = new_md.replace('</font><br>\n \n<font color = "#B22222"></font><br>  \n\n','</font><br>\n')
    new_md = new_md.replace('</font<br>\n<font color = "#B22222"> </font><br>\n','</font><br>\n')
    new_md = new_md.replace('</font><br>\n<font color = "#B22222"> </font><br>\n','</font><br>\n')
    # fix cases like for `fisher.test` on `b_10.html` where want `\n` in text and not `\\n` or a line break
    new_md = new_md.replace(r'"\\n"',r'"\n"') # `.replace('"\\\\n"',r'"\n"')` also works, i.e., double-escaping for find
    # fix a few cases of typos found
    if md == 'h_01.md':
        new_md = new_md.replace('require(lsmeans){','require(lsmeans)){') #addresses a current typo in first code block of Salvatore's `h_01.html`
    %store new_md > temp.txt 
    !mv temp.txt {md}
    sys.stderr.write("Non-breaking spaces, and spaces in results lines, substituted with placeholders earlier and\nleft up until now to avoid introducing errors have been replaced\nwith spaces in {}"
        ".\n".format(md))

In [ ]:
# Custom fixes for particular markdowns that don't conform for whatever reason
a_06orig1 ='''install.packages("psych")'''
a_06new1 ='''# NEXT LINE COMMENTED OUT BECAUSE WILL CAUSE HANG UP WHEN RUNNING AUTOMATED
#install.packages("psych")'''
a_06orig2 ='''```{.python .input}
D2 = read.table("male-female.dat", header=TRUE)
```

would read in data from a file called *male-female.dat*
found in the working directory.  In this case the file could be a
space-delimited text file:

 

```{.python .input}
Sex      Height

male     175

male     176

female   162

female   165
```
  

Or

 

```{.python .input}
D2 = read.table("male-female.csv", header=TRUE,
sep=",")
```
  

for a comma-separated file.

 

```{.python .input}
Sex,Height

male,175

male,176

female,162

female,165
```
  

```{.python .input}
D2
```
<font color = "#B22222">     Sex Height</font><br>
<font color = "#B22222">1   male    175</font><br>
<font color = "#B22222">2   male    176</font><br>
<font color = "#B22222">3 female    162</font><br>
<font color = "#B22222">4 female    165</font><br>
 

R Studio also has an easy interface in the *Tools* menu
to import data from a file.

 

The *getwd* function will show the location of the
working directory, and *setwd* can be used to set the working directory.

 

```{.python .input}
getwd()
```'''
a_06new2 ='''```R
# FILE MAKING ADDED SO THE NOTEBOOK WORKS FOR AUTOMATED RUN
Input =("
Sex     Height
male    175
male    176
female  162
female  165
")

fileConn<-file("male-female.dat")
writeLines(Input, fileConn)
close(fileConn) # write block based on https://stackoverflow.com/a/2470277/8508004
# END FILE MAKING ADDED SO THE NOTEBOOK WORKS FOR AUTOMATED RUN

D2 = read.table("male-female.dat", header=TRUE)
```

would read in data from a file called *male-female.dat*
found in the working directory.  In this case the file could be a
space-delimited text file:

 


Sex      Height

male     175

male     176

female   162

female   165

  

Or

for a comma-separated file.

 
Sex,Height

male,175

male,176

female,162

female,165


```R
# FILE MAKING ADDED SO THE NOTEBOOK WORKS FOR AUTOMATED RUN
Input =("
Sex,Height

male,175

male,176

female,162

female,165
")

fileConn<-file("male-female.csv")
writeLines(Input, fileConn)
close(fileConn) # write block based on https://stackoverflow.com/a/2470277/8508004
# END FILE MAKING ADDED SO THE NOTEBOOK WORKS FOR AUTOMATED RUN


D2 = read.table("male-female.csv", header=TRUE,
sep=",")

D2
```
  


  

<font color = "#B22222">     Sex Height</font><br>
<font color = "#B22222">1   male    175</font><br>
<font color = "#B22222">2   male    176</font><br>
<font color = "#B22222">3 female    162</font><br>
<font color = "#B22222">4 female    165</font><br>
 

R Studio also has an easy interface in the *Tools* menu
to import data from a file.

 

The *getwd* function will show the location of the
working directory, and *setwd* can be used to set the working directory.

 

```R
getwd()
```'''
a_06orig3 ='''```{.python .input}
setwd("C:/Users/Salvatore/Desktop")
```'''
a_06new3 ='''```R
# NEXT LINE COMMENTED OUT BECAUSE NEEDS CUSTOMIZING & WILL CAUSE HANG UP WHEN RUNNING AUTOMATED
#setwd("C:/Users/Salvatore/Desktop")
```'''
a_07orig1 ='''```{.python .input}
str(D1$ Sex)
```'''
a_07new1 ='''```R
# DATAFRAME MAKING FROM PREVIOUS PAGE ADDED SO THE NOTEBOOK WORKS FOR AUTOMATED RUN
Input =("
Sex     Height
male    175
male    176
female  162
female  165
")

D1 = read.table(textConnection(Input),header=TRUE)
# END ADDED DATAFRAME MAKING FOR AUTOMATED RUN

str(D1$ Sex)
```'''
a_08orig1 ='''RSiteSearch("chi-square")'''
a_08new1 ='''# NEXT LINE COMMENTED OUT BECAUSE WON'T WORK FROM BINDER & WILL CAUSE HANG UP WHEN RUNNING AUTOMATED
#RSiteSearch("chi-square")'''

d_05orig1 ='''### options(contrasts = c("contr.sum",
"contr.poly"))'''
d_05new1 ='''### options(contrasts = c("contr.sum","contr.poly"))'''
d_05orig2 ='''library(lsmeans) library(multcompView)'''
d_05new2 ='''library(lsmeans)
library(multcompView)'''
d_05orig3 ='''pairwise ~ Location,
                      adjust = "tukey")'''
d_05new3 ='''pairwise ~ Location,
                      adjust = "tukey")
summary(leastsquare)'''
d_05orig4 ='''cld(leastsquare,
    alpha   = 0.05,
    Letters = letters,
    adjust="tukey")'''
d_05new4 ='''library(emmeans)
library(multcomp)

em = emmeans(model,  ~ Location)

cld(em,
    alpha=0.05,
    Letters=letters,      ### Use lower-case letters for .group
    adjust="tukey")       ### Tukey-adjusted comparisons'''


d_06orig1 ='''```{.python .input}
PT = PT$res

PT

library(rcompanion)

cldList(comparison = PT$Comparison,
        p.value    =
PT$P.adj,
        threshold  = 0.05)
```'''
d_06new1 ='''#CODE BLOCK COMMENTED OFF BECAUSE CANNOT AUTOMATE RUN DUE TO `Error: No significant differences.` AS OUTPUT.
PT = PT$res

PT

library(rcompanion)

cldList(comparison = PT$Comparison,
        p.value    =
PT$P.adj,
        threshold  = 0.05)
        
'''

d_06orig2 ='''```{.python .input}
library(rcompanion)

cldList(comparison = PT$Comparison,
        p.value    =
PT$P.adj,
        threshold  = 0.05)
```'''
d_06new2 ='''#CODE BLOCK COMMENTED OFF BECAUSE CANNOT AUTOMATE RUN DUE TO `Error: No significant differences.` AS OUTPUT.
library(rcompanion)

cldList(comparison = PT$Comparison,
        p.value    =
PT$P.adj,
        threshold  = 0.05)

'''

d_07orig1 ='''```{.python .input}
### additional model checking plots
with: plot(model)
```'''
d_07new1 ='''### additional model checking plots

`plot(model)`'''

d_07orig2 ='''cld(leastsquare,
    alpha=0.05,
    Letters=letters,      ### Use lower-case letters for .group
    adjust="tukey")       ### Tukey-adjusted comparisons'''
d_07new2 ='''library(emmeans)
library(multcomp)

em = emmeans(model, ~ Tech)

cld(em,
    alpha=0.05,
    Letters=letters,      ### Use lower-case letters for .group
    adjust="tukey")       ### Tukey-adjusted comparisons'''
d_07orig3 ='''library(TukeyC)

tuk = TukeyC(Data,
             model = 'Protein ~ Tech + Error(Rat)',
             error = 'Rat',
             which = 'Tech',
             fl1=1,
             sig.level = 0.05)

summary(tuk)'''
d_07new3 ='''library(TukeyC)

tuk = TukeyC(Protein ~ Tech + Error(Rat),
             data = Data,
             which = 'Tech',
             error = 'Rat',)

summary(tuk)'''

d_08orig1 ='''lsmeans = lmerTest::lsmeans ### Uses the lsmeans function'''
d_08new1 ='''lsmeans = lmerTest::lsmeansLT ### Uses the lsmeans function'''
d_08orig2 ='''### Extract lsmeans table
Sum = PT$diffs.lsmeans.table


### Extract comparisons and p-values
  Comparison = rownames(Sum)

P.value   = Sum$'p-value'


### Adjust p-values
P.value.adj = p.adjust(P.value,
                       method =  "none"[)   

]{style="color:blue"}### Fix names of comparisons
Comparison = gsub("-"[, ]{style="color:blue"}"- Day"[, Comparison)


]{style="color:blue"}### Produce compact letter display
library(rcompanion)

cldList(comparison = Comparison,
        p.value    = P.value.adj,
        threshold = 0.05)'''
d_08new2 ='''### Extract lsmeans table
Sum = PT$diffs.lsmeans.table


### Extract comparisons and p-values
  Comparison = rownames(Sum)

P.value   = Sum$'p-value'


### Adjust p-values
P.value.adj = p.adjust(P.value,
                       method =  "none")   

### Fix names of comparisons
Comparison = gsub("-", "- Day", Comparison)


### Produce compact letter display
library(rcompanion)

cldList(comparison = Comparison,
        p.value    = P.value.adj,
        threshold = 0.05)'''
d_08orig3 ='''library(multcompView)
library(lsmeans)

lsmeans = lsmeans::lsmeans ### Uses the lsmeans  function                           
###  from the lsmeans package,
                            ###  not from the lmerTest package
leastsquare = lsmeans(model, 
                      pairwise ~ Sex:Genotype,
                      adjust="tukey")

 

cld(leastsquare, 
    alpha=.05,  
    Letters=letters)'''
d_08new3 ='''library(emmeans)
library(multcomp)

ieff = emmeans(model, ~ Sex:Genotype)

cld(ieff, alpha=.05, Letters=letters)'''
d_08orig4 ='''library(multcompView)
library(lsmeans)

lsmeans = lsmeans::lsmeans ### Uses the lsmeans function
                           ###  from the lsmeans package,
                           ###  not from the lmerTest package
leastsquare = lsmeans(model, 
                     pairwise ~ Day,
                     alpha=.05,
                     adjust="tukey")

cld(leastsquare, 
    alpha=.05,  
    Letters=letters,
    adjust="tukey")'''
d_08new4 ='''library(emmeans)
library(multcomp)

em = emmeans(model, ~ Day)

cld(em, alpha=.05, Letters=letters)'''

d_08aorig1 ='''```{.python .input}
### Order groups by median
 
Data$Factor.A = factor(Data$Factor.A, 
                     
levels = c("n", "l", "m"))


### Pairwise robust test
  library(rcompanion)

PT = pairwiseRobustTest(
     Response ~ Factor.A, 
     data = Data,
     est="mom", 
     nboot=5000,
     method="fdr")      # adjust p-values; see ?p.adjust for options
PT
```'''
d_08anew1 ='''
### Order groups by median
 
Data$Factor.A = factor(Data$Factor.A, 
                     
levels = c("n", "l", "m"))


### Pairwise robust test
  library(rcompanion)

PT = pairwiseRobustTest(
     Response ~ Factor.A, 
     data = Data,
     est="mom", 
     nboot=5000,
     method="fdr")      # adjust p-values; see ?p.adjust for options
PT
'''
d_08aorig2 ='''```{.python .input}
### Produce compact letter display 
library(rcompanion)

cldList(comparison = PT$Comparison,
        p.value    = PT$p.adjust,
        threshold  = 0.05)
```'''
d_08anew2 ='''
### Produce compact letter display 
library(rcompanion)

cldList(comparison = PT$Comparison,
        p.value    = PT$p.adjust,
        threshold  = 0.05)

'''
d_08aorig3 ='''```{.python .input}
library(rcompanion)

PT = pairwiseRobustTest(
     Response ~ Factor.int, 
     data = Data,
     est="mom", 
     nboot=5000,
     method="fdr")      # adjust p-values; see ?p.adjust for options
PT
```'''
d_08anew3 ='''
library(rcompanion)

PT = pairwiseRobustTest(
     Response ~ Factor.int, 
     data = Data,
     est="mom", 
     nboot=5000,
     method="fdr")      # adjust p-values; see ?p.adjust for options
PT

'''
d_08aorig5 ='''```{.python .input}
### Order groups by median
 
Data$Factor = factor(Data$Factor, 
                     
levels = c("n", "l", "m"))


### Pairwise robust tests
  library(rcompanion)

PM = pairwiseRobustMatrix(
     Response ~ Factor.A, 
     data = Data,
     est="mom", 
     nboot=5000,
     method="fdr")      # adjust p-values; see ?p.adjust for options
PM$Adjusted
```'''
d_08anew5 ='''
### Order groups by median
 
Data$Factor = factor(Data$Factor, 
                     
levels = c("n", "l", "m"))


### Pairwise robust tests
  library(rcompanion)

PM = pairwiseRobustMatrix(
     Response ~ Factor.A, 
     data = Data,
     est="mom", 
     nboot=5000,
     method="fdr")      # adjust p-values; see ?p.adjust for options
PM$Adjusted
'''
d_08aorig6 ='''```{.python .input}
library(multcompView)

multcompLetters(PM$Adjusted,
                compare="<",
                threshold=0.05,
                Letters=letters,
                reversed = FALSE)
```'''
d_08anew6 ='''
library(multcompView)

multcompLetters(PM$Adjusted,
                compare="<",
                threshold=0.05,
                Letters=letters,
                reversed = FALSE)

'''
d_08aorig7 ='''```{.python .input}
### Order groups by median
  Data$Factor.int = factor(Data$Factor.int,
                         
levels = c("m.y", "m.x", "n.x", "l.y", "n.y", "l.x"))


### Pairwise robust tests
  library(rcompanion)

PM = pairwiseRobustMatrix(
     Response ~ Factor.int, 
     data = Data,
     est="mom", 
     nboot=5000,
     method="fdr")      # adjust p-values; see ?p.adjust for options
PM
```'''
d_08anew7 ='''
### Order groups by median
  Data$Factor.int = factor(Data$Factor.int,
                         
levels = c("m.y", "m.x", "n.x", "l.y", "n.y", "l.x"))


### Pairwise robust tests
  library(rcompanion)

PM = pairwiseRobustMatrix(
     Response ~ Factor.int, 
     data = Data,
     est="mom", 
     nboot=5000,
     method="fdr")      # adjust p-values; see ?p.adjust for options
PM
'''

e_01orig1 ='''pwr.r.test(n = NULL, '''
e_01new1 ='''library(pwr)
pwr.r.test(n = NULL, '''

e_03orig1 ='''### additional model checking plots
with: plot(model.final)'''
e_03new1 ='''### additional model checking plots with:
plot(model.final)'''
e_03orig2 ='''### additional model checking plots
with: plot(model)'''
e_03new2 ='''### additional model checking plots with:
plot(model)'''

e_04orig1 ='''### additional model checking plots with: plot(model.2)
### alternative: library(FSA);
residPlot(model.2)'''
e_04new1 ='''### additional model checking plots with: 
plot(model.2)
### alternative: 
library(FSA);
residPlot(model.2)'''

e_05orig2 ='''compare.lm'''
e_05new2 ='''compareLM'''

e_06orig1 ='''### additional model checking plots
with: plot(model)'''
e_06new1 ='''### additional model checking plots with:
plot(model)'''

e_07orig1 ='''Species   Status Length
Mass Range Migr Insect Diet Clutch Broods Wood Upland Water Release Indiv
 Cyg_olor  1      1520  9600 1.21  1    12     2     6     1      0    0     
1      6        29
 Cyg_atra  1      1250  5000 0.56  1     0     1     6     1      0    0     
1     10        85
 Cer_nova  1       870  3360 0.07  1     0     1     4     1      0    0     
1      3         8
 Ans_caer  0       720  2517 1.1   3    12     2     3.8   1      0    0     
1      1        10
 Ans_anse  0       820  3170 3.45  3     0     1     5.9   1      0    0     
1      2         7
 Bra_cana  1       770  4390 2.96  2     0     1     5.9   1      0    0     
1     10        60
 Bra_sand  0        50  1930 0.01  1     0     1     4     2      0    0     
0      1         2
 Alo_aegy  0       680  2040 2.71  1    NA     2     8.5   1      0    0     
1      1         8
 Ana_plat  1       570  1020 9.01  2     6     2    12.6   1      0    0     
1     17      1539
 Ana_acut  0       580   910 7.9   3     6     2     8.3   1      0    0     
1      3       102
 Ana_pene  0       480   590 4.33  3     0     1     8.7   1      0    0     
1      5        32
 Aix_spon  0       470   539 1.04  3    12     2    13.5   2      1    0     
1      5        10
 Ayt_feri  0       450   940 2.17  3    12     2     9.5   1      0    0     
1      3         9
 Ayt_fuli  0       435   684 4.81  3    12     2    10.1   1      0    0     
1      2         5
 Ore_pict  0       275   230 0.31  1     3     1     9.5   1      1    1     
0      9       398
 Lop_cali  1       256   162 0.24  1     3     1    14.2   2      0    0     
0     15      1420
 Col_virg  1       230   170 0.77  1     3     1    13.7   1      0    0     
0     17      1156
 Ale_grae  1       330   501 2.23  1     3     1    15.5   1      0    1     
0     15       362
 Ale_rufa  0       330   439 0.22  1     3     2    11.2   2      0    0     
0      2        20
 Per_perd  0       300   386 2.4   1     3     1    14.6   1      0    1     
0     24       676
 Cot_pect  0       182    95 0.33  3    NA     2     7.5   1      0    0     
0      3      NA
 Cot_aust  1       180    95 0.69  2    12     2    11     1      0    0     
1     11       601
 Lop_nyct  0       800  1150 0.28  1    12     2     5     1      1    1     
0      4         6
 Pha_colc  1       710   850 1.25  1    12     2    11.8   1      1    0     
0     27       244
 Syr_reev  0       750   949 0.2   1    12     2     9.5   1      1    1     
0      2         9
 Tet_tetr  0       470   900 4.17  1     3     1     7.9   1      1    1     
0      2        13
 Lag_lago  0       390   517 7.29  1     0     1     7.5   1      1    1     
0      2         4
 Ped_phas  0       440   815 1.83  1     3     1    12.3   1      1    0     
0      1        22
 Tym_cupi  0       435   770 0.26  1     4     1    12     1      0    0     
0      3        57
 Van_vane  0       300   226 3.93  2    12     3     3.8   1      0    0     
0      8       124
 Plu_squa  0       285   318 1.67  3    12     3     4     1      0    0     
1      2         3
 Pte_alch  0       350   225 1.21  2     0     1     2.5   2      0    0     
0      1         8
 Pha_chal  0       320   350 0.6   1    12     2     2     2      1    0     
0      8        42
 Ocy_loph  0       330   205 0.76  1     0     1     2     7      1    0     
1      4        23
 Leu_mela  0       372  NA   0.07  1    12     2     2     1      1    0      0     
6        34
 Ath_noct  1       220   176 4.84  1    12     3     3.6   1      1    0     
0      7       221
 Tyt_alba  0       340   298 8.9   2     0     3     5.7   2      1    0     
0      1         7
 Dac_nova  1       460   382 0.34  1    12     3     2     1      1    0     
0      7        21
 Lul_arbo  0       150  32.1 1.78  2     4     2     3.9   2      1    0     
0      1         5
 Ala_arve  1       185  38.9 5.19  2    12     2     3.7   3      0    0     
0     11       391
 Pru_modu  1       145  20.5 1.95  2    12     2     3.4   2      1    0     
0     14       245
 Eri_rebe  0       140  15.8 2.31  2    12     2     5     2      1    0     
0     11       123
 Lus_mega  0       161  19.4 1.88  3    12     2     4.7   2      1    0     
0      4         7
 Tur_meru  1       255  82.6 3.3   2    12     2     3.8   3      1    0     
0     16       596
 Tur_phil  1       230  67.3 4.84  2    12     2     4.7   2      1    0     
0     12       343
 Syl_comm  0       140  12.8 3.39  3    12     2     4.6   2      1    0     
0      1         2
 Syl_atri  0       142  17.5 2.43  2     5     2     4.6   1      1    0     
0      1         5
 Man_mela  0       180  NA   0.04  1    12     3     1.9   5      1    0     
0      1         2
 Man_mela  0       265    59 0.25  1    12     2     2.6   NA     1    0     
0      1        80
 Gra_cyan  0       275   128 0.83  1    12     3     3     2      1    0     
1      1      NA
 Gym_tibi  1       400   380 0.82  1    12     3     4     1      1    0     
0     15       448
 Cor_mone  0       335   203 3.4   2    12     2     4.5   1      1    0     
0      2         3
 Cor_frug  1       400   425 3.73  1    12     2     3.6   1      1    0     
0     10       182 
 Stu_vulg  1       222  79.8 3.33  2     6     2     4.8   2      1    0     
0     14       653
 Acr_tris  1       230 111.3 0.56  1    12     2     3.7   1      1    0     
0      5        88
 Pas_dome  1       149  28.8 6.5   1     6     2     3.9   3      1    0     
0     12       416
 Pas_mont  0       133    22 6.8   1     6     2     4.7   3      1    0     
0      3        14
 Aeg_temp  0       120  NA   0.17  1     6     2     4.7   3      1    0     
0      3        14
 Emb_gutt  0       120    19 0.15  1     4     1     5     3      0    0     
0      4       112
 Poe_gutt  0       100  12.4 0.75  1     4     1     4.7   3      0    0     
0      1        12
 Lon_punc  0       110  13.5 1.06  1     0     1     5     3      0    0     
0      1         8
 Lon_cast  0       100  NA   0.13  1     4     1     5     NA     0    0     
1      4        45
 Pad_oryz  0       160  NA   0.09  1     0     1     5     NA     0    0     
0      2         6
 Fri_coel  1       160  23.5 2.61  2    12     2     4.9   2      1    0     
0     17       449
 Fri_mont  0       146  21.4 3.09  3    10     2     6     NA     1    0     
0      7       121
 Car_chlo  1       147  29   2.09  2     7     2     4.8   2      1    0     
0      6        65
 Car_spin  0       117  12   2.09  3     3     1     4     2      1    0     
0      3        54
 Car_card  1       120  15.5 2.85  2     4     1     4.4   3      1    0     
0     14       626
 Aca_flam  1       115  11.5 5.54  2     6     1     5     2      1    0     
0     10       607
 Aca_flavi 0       133  17   1.67  2     0     1     5     3      0    1     
0      3        61
 Aca_cann  0       136  18.5 2.52  2     6     1     4.7   2      1    0     
0     12       209
 Pyr_pyrr  0       142  23.5 3.57  1     4     1     4     3      1    0      0     
2      NA
 Emb_citr  1       160  28.2 4.11  2     8     2     3.3   3      1    0     
0     14       656
 Emb_hort  0       163  21.6 2.75  3    12     2     5     1      0    0     
0      1         6
 Emb_cirl  1       160  23.6 0.62  1    12     2     3.5   2      1    0     
0      3        29
 Emb_scho  0       150  20.7 5.42  1    12     2     5.1   2      0    0     
1      2         9
 Pir_rubr  0       170  31   0.55  3    12     2     4     NA     1    0     
0      1         2
 Age_phoe  0       210  36.9 2     2     8     2     3.7   1      0    0     
1      1         2
 Stu_negl  0       225 106.5 1.2   2    12     2     4.8   2      0    0     
0      1         2'''
e_07new1 ='''Species   Status Length Mass Range Migr Insect Diet Clutch Broods Wood Upland Water Release Indiv
 Cyg_olor  1      1520  9600 1.21  1    12     2     6     1      0    0      1      6        29
 Cyg_atra  1      1250  5000 0.56  1     0     1     6     1      0    0      1     10        85
 Cer_nova  1       870  3360 0.07  1     0     1     4     1      0    0      1      3         8
 Ans_caer  0       720  2517 1.1   3    12     2     3.8   1      0    0      1      1        10
 Ans_anse  0       820  3170 3.45  3     0     1     5.9   1      0    0      1      2         7
 Bra_cana  1       770  4390 2.96  2     0     1     5.9   1      0    0      1     10        60
 Bra_sand  0        50  1930 0.01  1     0     1     4     2      0    0      0      1         2
 Alo_aegy  0       680  2040 2.71  1    NA     2     8.5   1      0    0      1      1         8
 Ana_plat  1       570  1020 9.01  2     6     2    12.6   1      0    0      1     17      1539
 Ana_acut  0       580   910 7.9   3     6     2     8.3   1      0    0      1      3       102
 Ana_pene  0       480   590 4.33  3     0     1     8.7   1      0    0      1      5        32
 Aix_spon  0       470   539 1.04  3    12     2    13.5   2      1    0      1      5        10
 Ayt_feri  0       450   940 2.17  3    12     2     9.5   1      0    0      1      3         9
 Ayt_fuli  0       435   684 4.81  3    12     2    10.1   1      0    0      1      2         5
 Ore_pict  0       275   230 0.31  1     3     1     9.5   1      1    1      0      9       398
 Lop_cali  1       256   162 0.24  1     3     1    14.2   2      0    0      0     15      1420
 Col_virg  1       230   170 0.77  1     3     1    13.7   1      0    0      0     17      1156
 Ale_grae  1       330   501 2.23  1     3     1    15.5   1      0    1      0     15       362
 Ale_rufa  0       330   439 0.22  1     3     2    11.2   2      0    0      0      2        20
 Per_perd  0       300   386 2.4   1     3     1    14.6   1      0    1      0     24       676
 Cot_pect  0       182    95 0.33  3    NA     2     7.5   1      0    0      0      3      NA
 Cot_aust  1       180    95 0.69  2    12     2    11     1      0    0      1     11       601
 Lop_nyct  0       800  1150 0.28  1    12     2     5     1      1    1      0      4         6
 Pha_colc  1       710   850 1.25  1    12     2    11.8   1      1    0      0     27       244
 Syr_reev  0       750   949 0.2   1    12     2     9.5   1      1    1      0      2         9
 Tet_tetr  0       470   900 4.17  1     3     1     7.9   1      1    1      0      2        13
 Lag_lago  0       390   517 7.29  1     0     1     7.5   1      1    1      0      2         4
 Ped_phas  0       440   815 1.83  1     3     1    12.3   1      1    0      0      1        22
 Tym_cupi  0       435   770 0.26  1     4     1    12     1      0    0      0      3        57
 Van_vane  0       300   226 3.93  2    12     3     3.8   1      0    0      0      8       124
 Plu_squa  0       285   318 1.67  3    12     3     4     1      0    0      1      2         3
 Pte_alch  0       350   225 1.21  2     0     1     2.5   2      0    0      0      1         8
 Pha_chal  0       320   350 0.6   1    12     2     2     2      1    0      0      8        42
 Ocy_loph  0       330   205 0.76  1     0     1     2     7      1    0      1      4        23
 Leu_mela  0       372  NA   0.07  1    12     2     2     1      1    0      0      6        34
 Ath_noct  1       220   176 4.84  1    12     3     3.6   1      1    0      0      7       221
 Tyt_alba  0       340   298 8.9   2     0     3     5.7   2      1    0      0      1         7
 Dac_nova  1       460   382 0.34  1    12     3     2     1      1    0      0      7        21
 Lul_arbo  0       150  32.1 1.78  2     4     2     3.9   2      1    0      0      1         5
 Ala_arve  1       185  38.9 5.19  2    12     2     3.7   3      0    0      0     11       391
 Pru_modu  1       145  20.5 1.95  2    12     2     3.4   2      1    0      0     14       245
 Eri_rebe  0       140  15.8 2.31  2    12     2     5     2      1    0      0     11       123
 Lus_mega  0       161  19.4 1.88  3    12     2     4.7   2      1    0      0      4         7
 Tur_meru  1       255  82.6 3.3   2    12     2     3.8   3      1    0      0     16       596
 Tur_phil  1       230  67.3 4.84  2    12     2     4.7   2      1    0      0     12       343
 Syl_comm  0       140  12.8 3.39  3    12     2     4.6   2      1    0      0      1         2
 Syl_atri  0       142  17.5 2.43  2     5     2     4.6   1      1    0      0      1         5
 Man_mela  0       180  NA   0.04  1    12     3     1.9   5      1    0      0      1         2
 Man_mela  0       265    59 0.25  1    12     2     2.6   NA     1    0      0      1        80
 Gra_cyan  0       275   128 0.83  1    12     3     3     2      1    0      1      1      NA
 Gym_tibi  1       400   380 0.82  1    12     3     4     1      1    0      0     15       448
 Cor_mone  0       335   203 3.4   2    12     2     4.5   1      1    0      0      2         3
 Cor_frug  1       400   425 3.73  1    12     2     3.6   1      1    0      0     10       182 
 Stu_vulg  1       222  79.8 3.33  2     6     2     4.8   2      1    0      0     14       653
 Acr_tris  1       230 111.3 0.56  1    12     2     3.7   1      1    0      0      5        88
 Pas_dome  1       149  28.8 6.5   1     6     2     3.9   3      1    0      0     12       416
 Pas_mont  0       133    22 6.8   1     6     2     4.7   3      1    0      0      3        14
 Aeg_temp  0       120  NA   0.17  1     6     2     4.7   3      1    0      0      3        14
 Emb_gutt  0       120    19 0.15  1     4     1     5     3      0    0      0      4       112
 Poe_gutt  0       100  12.4 0.75  1     4     1     4.7   3      0    0      0      1        12
 Lon_punc  0       110  13.5 1.06  1     0     1     5     3      0    0      0      1         8
 Lon_cast  0       100  NA   0.13  1     4     1     5     NA     0    0      1      4        45
 Pad_oryz  0       160  NA   0.09  1     0     1     5     NA     0    0      0      2         6
 Fri_coel  1       160  23.5 2.61  2    12     2     4.9   2      1    0      0     17       449
 Fri_mont  0       146  21.4 3.09  3    10     2     6     NA     1    0      0      7       121
 Car_chlo  1       147  29   2.09  2     7     2     4.8   2      1    0      0      6        65
 Car_spin  0       117  12   2.09  3     3     1     4     2      1    0      0      3        54
 Car_card  1       120  15.5 2.85  2     4     1     4.4   3      1    0      0     14       626
 Aca_flam  1       115  11.5 5.54  2     6     1     5     2      1    0      0     10       607
 Aca_flavi 0       133  17   1.67  2     0     1     5     3      0    1      0      3        61
 Aca_cann  0       136  18.5 2.52  2     6     1     4.7   2      1    0      0     12       209
 Pyr_pyrr  0       142  23.5 3.57  1     4     1     4     3      1    0      0      2      NA
 Emb_citr  1       160  28.2 4.11  2     8     2     3.3   3      1    0      0     14       656
 Emb_hort  0       163  21.6 2.75  3    12     2     5     1      0    0      0      1         6
 Emb_cirl  1       160  23.6 0.62  1    12     2     3.5   2      1    0      0      3        29
 Emb_scho  0       150  20.7 5.42  1    12     2     5.1   2      0    0      1      2         9
 Pir_rubr  0       170  31   0.55  3    12     2     4     NA     1    0      0      1         2
 Age_phoe  0       210  36.9 2     2     8     2     3.7   1      0    0      1      1         2
 Stu_negl  0       225 106.5 1.2   2    12     2     4.8   2      0    0      0      1         2'''
e_07orig2 = '''Species   Status Length
Mass Range Migr Insect Diet Clutch Broods Wood Upland Water Release Indiv
 Cyg_olor  1      1520  9600 1.21  1    12     2     6     1      0    0     
1      6        29
 Cyg_atra  1      1250  5000 0.56  1     0     1     6     1      0    0     
1     10        85
 Cer_nova  1       870  3360 0.07  1     0     1     4     1      0    0     
1      3         8
 Ans_caer  0       720  2517 1.1   3    12     2     3.8   1      0    0     
1      1        10
 Ans_anse  0       820  3170 3.45  3     0     1     5.9   1      0    0     
1      2         7
 Bra_cana  1       770  4390 2.96  2     0     1     5.9   1      0    0     
1     10        60
 Bra_sand  0        50  1930 0.01  1     0     1     4     2      0    0     
0      1         2
 Alo_aegy  0       680  2040 2.71  1    NA     2     8.5   1      0    0     
1      1         8
 Ana_plat  1       570  1020 9.01  2     6     2    12.6   1      0    0     
1     17      1539
 Ana_acut  0       580   910 7.9   3     6     2     8.3   1      0    0     
1      3       102
 Ana_pene  0       480   590 4.33  3     0     1     8.7   1      0    0     
1      5        32
 Aix_spon  0       470   539 1.04  3    12     2    13.5   2      1    0     
1      5        10
 Ayt_feri  0       450   940 2.17  3    12     2     9.5   1      0    0     
1      3         9
 Ayt_fuli  0       435   684 4.81  3    12     2    10.1   1      0    0     
1      2         5
 Ore_pict  0       275   230 0.31  1     3     1     9.5   1      1    1     
0      9       398
 Lop_cali  1       256   162 0.24  1     3     1    14.2   2      0    0     
0     15      1420
 Col_virg  1       230   170 0.77  1     3     1    13.7   1      0    0      0    
17      1156
 Ale_grae  1       330   501 2.23  1     3     1    15.5   1      0    1     
0     15       362
 Ale_rufa  0       330   439 0.22  1     3     2    11.2   2      0    0     
0      2        20
 Per_perd  0       300   386 2.4   1     3     1    14.6   1      0    1     
0     24       676
 Cot_pect  0       182    95 0.33  3    NA     2     7.5   1      0    0     
0      3      NA
 Cot_aust  1       180    95 0.69  2    12     2    11     1      0    0     
1     11       601
 Lop_nyct  0       800  1150 0.28  1    12     2     5     1      1    1     
0      4         6
 Pha_colc  1       710   850 1.25  1    12     2    11.8   1      1    0     
0     27       244
 Syr_reev  0       750   949 0.2   1    12     2     9.5   1      1    1     
0      2         9
 Tet_tetr  0       470   900 4.17  1     3     1     7.9   1      1    1     
0      2        13
 Lag_lago  0       390   517 7.29  1     0     1     7.5   1      1    1     
0      2         4
 Ped_phas  0       440   815 1.83  1     3     1    12.3   1      1    0     
0      1        22
 Tym_cupi  0       435   770 0.26  1     4     1    12     1      0    0     
0      3        57
 Van_vane  0       300   226 3.93  2    12     3     3.8   1      0    0     
0      8       124
 Plu_squa  0       285   318 1.67  3    12     3     4     1      0    0     
1      2         3
 Pte_alch  0       350   225 1.21  2     0     1     2.5   2      0    0     
0      1         8
 Pha_chal  0       320   350 0.6   1    12     2     2     2      1    0     
0      8        42
 Ocy_loph  0       330   205 0.76  1     0     1     2     7      1    0     
1      4        23
 Leu_mela  0       372  NA   0.07  1    12     2     2     1      1    0     
0      6        34
 Ath_noct  1       220   176 4.84  1    12     3     3.6   1      1    0     
0      7       221
 Tyt_alba  0       340   298 8.9   2     0     3     5.7   2      1    0     
0      1         7
 Dac_nova  1       460   382 0.34  1    12     3     2     1      1    0     
0      7        21
 Lul_arbo  0       150  32.1 1.78  2     4     2     3.9   2      1    0     
0      1         5
 Ala_arve  1       185  38.9 5.19  2    12     2     3.7   3      0    0     
0     11       391
 Pru_modu  1       145  20.5 1.95  2    12     2     3.4   2      1    0     
0     14       245
 Eri_rebe  0       140  15.8 2.31  2    12     2     5     2      1    0     
0     11       123
 Lus_mega  0       161  19.4 1.88  3    12     2     4.7   2      1    0     
0      4         7
 Tur_meru  1       255  82.6 3.3   2    12     2     3.8   3      1    0     
0     16       596
 Tur_phil  1       230  67.3 4.84  2    12     2     4.7   2      1    0     
0     12       343
 Syl_comm  0       140  12.8 3.39  3    12     2     4.6   2      1    0     
0      1         2
 Syl_atri  0       142  17.5 2.43  2     5     2     4.6   1      1    0     
0      1         5
 Man_mela  0       180  NA   0.04  1    12     3     1.9   5      1    0     
0      1         2
 Man_mela  0       265    59 0.25  1    12     2     2.6   NA     1    0     
0      1        80
 Gra_cyan  0       275   128 0.83  1    12     3     3     2      1    0     
1      1      NA
 Gym_tibi  1       400   380 0.82  1    12     3     4     1      1    0     
0     15       448
 Cor_mone  0       335   203 3.4   2    12     2     4.5   1      1    0     
0      2         3
 Cor_frug  1       400   425 3.73  1    12     2     3.6   1      1    0     
0     10       182 
 Stu_vulg  1       222  79.8 3.33  2     6     2     4.8   2      1    0      0    
14       653
 Acr_tris  1       230 111.3 0.56  1    12     2     3.7   1      1    0     
0      5        88
 Pas_dome  1       149  28.8 6.5   1     6     2     3.9   3      1    0     
0     12       416
 Pas_mont  0       133    22 6.8   1     6     2     4.7   3      1    0     
0      3        14
 Aeg_temp  0       120  NA   0.17  1     6     2     4.7   3      1    0     
0      3        14
 Emb_gutt  0       120    19 0.15  1     4     1     5     3      0    0     
0      4       112
 Poe_gutt  0       100  12.4 0.75  1     4     1     4.7   3      0    0     
0      1        12
 Lon_punc  0       110  13.5 1.06  1     0     1     5     3      0    0     
0      1         8
 Lon_cast  0       100  NA   0.13  1     4     1     5     NA     0    0     
1      4        45
 Pad_oryz  0       160  NA   0.09  1     0     1     5     NA     0    0     
0      2         6
 Fri_coel  1       160  23.5 2.61  2    12     2     4.9   2      1    0     
0     17       449
 Fri_mont  0       146  21.4 3.09  3    10     2     6     NA     1    0     
0      7       121
 Car_chlo  1       147  29   2.09  2     7     2     4.8   2      1    0     
0      6        65
 Car_spin  0       117  12   2.09  3     3     1     4     2      1    0     
0      3        54
 Car_card  1       120  15.5 2.85  2     4     1     4.4   3      1    0     
0     14       626
 Aca_flam  1       115  11.5 5.54  2     6     1     5     2      1    0     
0     10       607
 Aca_flavi 0       133  17   1.67  2     0     1     5     3      0    1     
0      3        61
 Aca_cann  0       136  18.5 2.52  2     6     1     4.7   2      1    0     
0     12       209
 Pyr_pyrr  0       142  23.5 3.57  1     4     1     4     3      1    0     
0      2      NA
 Emb_citr  1       160  28.2 4.11  2     8     2     3.3   3      1    0     
0     14       656
 Emb_hort  0       163  21.6 2.75  3    12     2     5     1      0    0     
0      1         6
 Emb_cirl  1       160  23.6 0.62  1    12     2     3.5   2      1    0     
0      3        29
 Emb_scho  0       150  20.7 5.42  1    12     2     5.1   2      0    0     
1      2         9
 Pir_rubr  0       170  31   0.55  3    12     2     4     NA     1    0     
0      1         2
 Age_phoe  0       210  36.9 2     2     8     2     3.7   1      0    0     
1      1         2
 Stu_negl  0       225 106.5 1.2   2    12     2     4.8   2      0    0     
0      1         2'''

e_07orig3 ='''summary(model)'''
e_07new3 ='''summary(model.final)'''

h_01orig1='''```{.python .input}
Input = "
Contrast          Merlot  Cabernet  Syrah  Chardonnay  Riesling 
Gewürtztraminer'''

h_01new1='''```{.python .input}
Input = "
Contrast          Merlot  Cabernet  Syrah  Chardonnay  Riesling  Gewürtztraminer'''
h_01orig2='''"

 names match the order of levels of the treatment variable
   ### The coefficients of each row sum to 0

Matriz'''

h_01new2='''"

   ### The column names match the order of levels of the treatment variable
   ### The coefficients of each row sum to 0

Matriz'''


repaird_dict = {"a_06.md":[[a_06orig1,a_06new1],[a_06orig2,a_06new2],[a_06orig3,a_06new3]],
                "a_07.md":[[a_07orig1,a_07new1],],
                "a_08.md":[[a_08orig1,a_08new1],],
                "b_09.md":[[']{style="color:#006600"}','']],
                "d_05.md":[[d_05orig1,d_05new1],[d_05orig2,d_05new2],[d_05orig3,d_05new3],[d_05orig4,d_05new4]],
                "d_06.md":[[d_06orig1,d_06new1],[d_06orig2,d_06new2]],
                "d_07.md":[[d_07orig1,d_07new1],[d_07orig2,d_07new2],[d_07orig3,d_07new3],],
                "d_08.md":[[d_08orig1,d_08new1],[d_08orig2,d_08new2],[d_08orig3,d_08new3],[d_08orig4,d_08new4]],
                "d_08a.md":[[d_08aorig1,d_08anew1],[d_08aorig2,d_08anew2],[d_08aorig3,d_08anew3],[d_08aorig5,d_08anew5],[d_08aorig6,d_08anew6],[d_08aorig7,d_08anew7],],
                "e_01.md":[[e_01orig1,e_01new1],],
                "e_03.md":[[e_03orig1,e_03new1],[e_03orig2,e_03new2]],
                "e_04.md":[[e_04orig1,e_04new1],],
                "e_05.md":[[e_03orig1,e_03new1],[e_05orig2,e_05new2],],
                "e_06.md":[[e_06orig1,e_06new1],],
                "e_07.md":[[e_07orig1,e_07new1],[e_07orig2,e_07new1],[e_07orig3,e_07new3]],
                "h_01.md":[[h_01orig1,h_01new1],[h_01orig2,h_01new2]]}

for md in repaird_dict:
    with open(md, 'r') as input:
        all_md=input.read()
    for f in repaird_dict[md]:
        if f[0] in all_md:
            all_md = all_md.replace(f[0],f[1])
        else:
            sys.stderr.write(f"Issue involving text `{f[0]}` seems no longer an "
                f"issue in `{md}`\nRemove the special handling for this.")

    %store all_md > {md}

## STEP #5: Start it cycling through markdown files making the output notebooks (unrun)

Because notedown makes the notebook, they won't be run. However, this allows the markdown generated earlier actually show as markdown in markdown cells that can be further edited, which I couldn't get from my earlier effort with Papermill alone.

In [ ]:
RCompanion_notebooks = []
for md in RCompanion_markdowns:
    notebook_name = f'{md.rsplit(".md",1)[0]}.ipynb'
    !notedown --match=fenced {md} > {notebook_name}
    RCompanion_notebooks .append(notebook_name)
    # LATER USE nbconvert to run edited versions of the notebooks in repo where to be located
    #!jupyter nbconvert --to notebook --execute {pair[0]} --output={pair[0]}
    #nbconvert use based on
    # https://nbconvert.readthedocs.io/en/latest/execute_api.html ; learned of
    # `output=` from usage by running `!jupyter nbconvert` in a Jupyter cell.
    # Otherwise, it was add `nbconvert` to file names in front of extension,
    #  like so: `Drawing_ideograms_Part1.nbconvert.ipynb`.
    '''
    !papermill Papermill\ Template\ for\ Circos\ later\ tutorial\ notebooks.ipynb {pair[0]} -p markdown {markdown} -p section {section} -p sub_pg {sub_pg} -p end_nb {end_nb} -p next_nb {next_nb}
    '''
    sys.stderr.write("\n*********************************************************\n"
        "Unrun notebook `{}` generated."
        "\n********************************************************"
        "*\n".format(notebook_name))

To follow up that, the notebooks each will later be run en masse [using nbconvert](https://nbconvert.readthedocs.io/en/latest/execute_api.html#executing-notebooks-from-the-command-line) to highlight if there are issues with individual ones.

Collecting all the results
-----------------------------------

Run the next cell to gather and archive both the produced initial draft `.ipynb` files and the edited markdown files.  
Also collect the three lists used here as json files so the contents can be used for automating filling in the markdown into Jupyter ipynb files.

In [ ]:
archive_file_name = "unrun_notebooks_from_RCompanion.tar.gz"
import os
import sys
# store `RCompanion_urls` and `RCompanion_markdowns` and `RCompanion_notebooks`
# as json since lighter-weight and more portable than pickling
# and may be useful in later steps if automate some post-generation editing.
RCompanion_urls_to_get_storedfn = "RCompanion_urls_to_get.json"
RCompanion_markdowns_made_storedfn = "RCompanion_markdowns_made.json"
RCompanion_notebooks_made_storedfn = "RCompanion_notebooks_made.json"

import json
with open(RCompanion_urls_to_get_storedfn, 'w') as f:
    json.dump(RCompanion_urls, f)
with open(RCompanion_markdowns_made_storedfn, 'w') as f:
    json.dump(RCompanion_markdowns, f)
with open(RCompanion_notebooks_made_storedfn, 'w') as f:
    json.dump(RCompanion_notebooks, f)
files_to_archive = RCompanion_notebooks + RCompanion_markdowns + [RCompanion_urls_to_get_storedfn] + [RCompanion_markdowns_made_storedfn] + [RCompanion_notebooks_made_storedfn]
!tar czf {archive_file_name} {" ".join(files_to_archive)}
sys.stderr.write("***************************DONE***********************************\n"
    "'{}' generated. Download it.\n"
    "***************************DONE***********************************".format(archive_file_name))

**Get organized**, now that everything has been used and processed (mostly).  
Move the individual files to sub-directories.

In [ ]:
!mkdir rcomp_html
!mkdir rcomp_mds
!mkdir rcomp_nbs

def extract_name_of_the_html(url, add_html_extension):
    '''
    make a file name based on the URL "https://rcompanion.org/rcompanion/index.html".
    if `add_html_extension` is True than add `.html` extension
    to the file name.
    
    Return filename
    '''
    split_url = url.split("/")
    fn = split_url[-1]
    if add_html_extension:
        fn += ".html"
    return fn


for url in RCompanion_urls:
    fn = extract_name_of_the_html(url, add_html_extension=False)
    !mv {fn} rcomp_html/.
for fn in RCompanion_markdowns:
    !mv {fn} rcomp_mds/.
for fn in RCompanion_notebooks:
    !mv {fn} rcomp_nbs/.

----

----

## Post-processing

Go on to three.


----

----